# Tensorflow single logistic

In [1]:
import numpy as np
import tensorflow as tf

C:\Users\d822u-r012\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
tf.set_random_seed(0)

In [16]:
w = tf.Variable(tf.zeros([2,1]))
b = tf.Variable(0.0, name="bias")

In [17]:
X = np.array([[0,0],[0,1],[1,0],[1,1]]) 
Y = np.array([[0],[1],[1],[1]])

#構造
x = tf.placeholder(tf.float32, shape=[None,2])
t = tf.placeholder(tf.float32, shape=[None,1])
y = tf.nn.sigmoid(tf.matmul(x,w)+b)

cross_entropy = tf.reduce_mean(tf.reduce_sum(-t * tf.log(y)-(1-t)*tf.log(1-y)))

optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

correct_prediction = tf.equal(tf.to_float(tf.greater(y,0.5)), t)

In [18]:
#初期化
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#学習
for epoch in range(200):
    sess.run(
        optimizer,
        feed_dict={
         x: X,   
         t: Y
        })

In [19]:
#結果
classified = correct_prediction.eval(session=sess,feed_dict={
    x: X,
    t: Y
})

In [20]:
prob = y.eval(session = sess, feed_dict={
    x:X
})

In [21]:
print("classified:")
print(classified)
print()
print("output probability:")
print(prob)

classified:
[[ True]
 [ True]
 [ True]
 [ True]]

output probability:
[[0.22355042]
 [0.9142595 ]
 [0.9142595 ]
 [0.99747413]]


# Keras single logistic

In [9]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.optimizers import SGD

Using TensorFlow backend.


In [10]:
np.random.seed(0)

In [11]:
model = Sequential()
model.add(Dense(input_dim=2, units=1))
model.add(Activation("sigmoid"))

In [12]:
model.compile(loss="binary_crossentropy",optimizer=SGD(0.1))

In [13]:
X = np.array([[0,0],[0,1],[1,0],[1,1]]) 
Y = np.array([[0],[1],[1],[1]])

In [14]:
model.fit(X,Y,epochs=200,batch_size=1)

Epoch 1/200
4/4 [==============================] - 0s 43ms/step - loss: 0.6378
Epoch 2/200
4/4 [==============================] - 0s 3ms/step - loss: 0.5876
Epoch 3/200
4/4 [==============================] - 0s 3ms/step - loss: 0.5494
Epoch 4/200
4/4 [==============================] - 0s 5ms/step - loss: 0.5195
Epoch 5/200
4/4 [==============================] - 0s 5ms/step - loss: 0.4957
Epoch 6/200
4/4 [==============================] - 0s 3ms/step - loss: 0.4763
Epoch 7/200
4/4 [==============================] - 0s 4ms/step - loss: 0.4609
Epoch 8/200
4/4 [==============================] - 0s 3ms/step - loss: 0.4480
Epoch 9/200
4/4 [==============================] - 0s 3ms/step - loss: 0.4363
Epoch 10/200
4/4 [==============================] - 0s 4ms/step - loss: 0.4262
Epoch 11/200
4/4 [==============================] - 0s 4ms/step - loss: 0.4176
Epoch 12/200
4/4 [==============================] - 0s 4ms/step - loss: 0.4095
Epoch 13/200
4/4 [==============================] - 0s 3ms/s

In [15]:
classes = model.predict_classes(X,batch_size=1)
prob = model.predict_proba(X,batch_size=1)

In [16]:
print("classified:")
print(Y==classes)
print()
print("output probability:")
print(prob)

classified:
[[ True]
 [ True]
 [ True]
 [ True]]

output probability:
[[0.21971276]
 [0.9121996 ]
 [0.9189271 ]
 [0.9976146 ]]


# TensorFlow Multi-Logstic

In [59]:
from sklearn.utils import shuffle

In [60]:
M = 2
K = 3
n = 100
N = n*K

In [61]:
X1 = np.random.randn(n,M) + np.array([0,10])
X2 = np.random.randn(n,M) + np.array([5,5])
X3 = np.random.randn(n,M) + np.array([10,0])
Y1 = np.array([[1,0,0] for i in range(n)])
Y2 = np.array([[0,1,0] for i in range(n)])
Y3 = np.array([[0,0,1] for i in range(n)])

X = np.concatenate((X1,X2,X3), axis = 0)
Y = np.concatenate((Y1,Y2,Y3), axis = 0)

In [62]:
#構造
W = tf.Variable(tf.zeros([M,K]))
b = tf.Variable(tf.zeros([K]))

x = tf.placeholder(tf.float32, shape=[None, M])
t = tf.placeholder(tf.float32, shape=[None, K])
y = tf.nn.softmax(tf.matmul(x,W) + b)

In [63]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(t*tf.log(y),reduction_indices=[1]))

In [64]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [65]:
batch_size = 50
n_batchs = N

In [66]:
#初期化
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(20):
    X_,Y_ = shuffle(X,Y)
    
    for i in range(n_batchs):
        start = i*batch_size
        end = start+batch_size
        
        sess.run(
        train_step,
        feed_dict=
            {
          x : X_[start:end],
          t : Y_[start:end]      
            }
        )

In [76]:
correct_prediction = tf.equal(tf.arg_max(y,1), tf.arg_max(t,1))

classified = correct_prediction.eval(session = sess, feed_dict={
    x : X_[0:10],
    t : Y_[0:10]
})

prob = y.eval(session = sess, feed_dict={
    x: X_[0:10]
})

print("classified:")
print(classified)
print()
print("output probability:")
print(prob)

classified:
[ True  True  True  True  True  True  True  True  True  True]

output probability:
[[8.00826996e-08 4.52177711e-02 9.54782128e-01]
 [8.31844531e-08 8.23544711e-03 9.91764486e-01]
 [4.95171786e-11 4.08923370e-04 9.99591053e-01]
 [1.37089512e-06 3.54836397e-02 9.64515030e-01]
 [1.05857785e-08 2.57217549e-02 9.74278271e-01]
 [5.89825511e-02 9.40316856e-01 7.00626755e-04]
 [6.14050142e-02 9.29098666e-01 9.49627906e-03]
 [9.70785499e-01 2.92145275e-02 4.09793834e-08]
 [2.32651051e-10 6.72258763e-03 9.93277371e-01]
 [8.90807033e-01 1.09192468e-01 3.73676471e-07]]


# Keras Multi-Logstic

In [35]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.optimizers import SGD

In [36]:
model = Sequential()
model.add(Dense(input_dim = M, units= K))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy",optimizer=SGD(lr=0.1))

In [37]:
minibatch_size = 50
model.fit(X,Y, epochs=200, batch_size = minibatch_size)

Epoch 1/200
300/300 [==============================] - 0s 423us/step - loss: 2.3342
Epoch 2/200
300/300 [==============================] - 0s 47us/step - loss: 0.1992
Epoch 3/200
300/300 [==============================] - 0s 63us/step - loss: 0.1392
Epoch 4/200
300/300 [==============================] - 0s 40us/step - loss: 0.1118
Epoch 5/200
300/300 [==============================] - 0s 60us/step - loss: 0.0943
Epoch 6/200
300/300 [==============================] - 0s 70us/step - loss: 0.0840
Epoch 7/200
300/300 [==============================] - 0s 37us/step - loss: 0.0750
Epoch 8/200
300/300 [==============================] - 0s 70us/step - loss: 0.0668
Epoch 9/200
300/300 [==============================] - 0s 37us/step - loss: 0.0610
Epoch 10/200
300/300 [==============================] - 0s 60us/step - loss: 0.0573
Epoch 11/200
300/300 [==============================] - 0s 50us/step - loss: 0.0534
Epoch 12/200
300/300 [==============================] - 0s 50us/step - loss: 0.0503


300/300 [==============================] - 0s 53us/step - loss: 0.0150
Epoch 99/200
300/300 [==============================] - 0s 47us/step - loss: 0.0148
Epoch 100/200
300/300 [==============================] - 0s 43us/step - loss: 0.0150
Epoch 101/200
300/300 [==============================] - 0s 37us/step - loss: 0.0148
Epoch 102/200
300/300 [==============================] - 0s 47us/step - loss: 0.0146
Epoch 103/200
300/300 [==============================] - 0s 50us/step - loss: 0.0148
Epoch 104/200
300/300 [==============================] - 0s 40us/step - loss: 0.0147
Epoch 105/200
300/300 [==============================] - 0s 37us/step - loss: 0.0145
Epoch 106/200
300/300 [==============================] - 0s 60us/step - loss: 0.0145
Epoch 107/200
300/300 [==============================] - 0s 50us/step - loss: 0.0144
Epoch 108/200
300/300 [==============================] - 0s 70us/step - loss: 0.0142
Epoch 109/200
300/300 [==============================] - 0s 63us/step - loss: 0.

300/300 [==============================] - 0s 64us/step - loss: 0.0112
Epoch 195/200
300/300 [==============================] - 0s 50us/step - loss: 0.0112
Epoch 196/200
300/300 [==============================] - 0s 37us/step - loss: 0.0112
Epoch 197/200
300/300 [==============================] - 0s 50us/step - loss: 0.0111
Epoch 198/200
300/300 [==============================] - 0s 60us/step - loss: 0.0108
Epoch 199/200
300/300 [==============================] - 0s 40us/step - loss: 0.0111
Epoch 200/200
300/300 [==============================] - 0s 47us/step - loss: 0.0108


In [38]:
X_,Y_ = shuffle(X,Y)
classes = model.predict_classes(X_[0:10], batch_size=minibatch_size)
prob = model.predict_proba(X_[0:10], batch_size=1)

In [39]:
print("classified:")
print(np.argmax(Y_[0:10],axis=1)== classes)
print()
print("output probability:")
print(prob)

classified:
[ True  True  True  True  True  True  True  True  True  True]

output probability:
[[2.4466121e-16 1.3933124e-04 9.9986064e-01]
 [2.0729772e-04 9.8363042e-01 1.6162233e-02]
 [9.9995458e-01 4.5394274e-05 4.4010781e-16]
 [9.9941611e-01 5.8389589e-04 1.3446563e-11]
 [4.7107835e-05 9.9675190e-01 3.2010989e-03]
 [2.4887710e-04 9.9934047e-01 4.1062414e-04]
 [1.7422870e-16 8.7280991e-05 9.9991274e-01]
 [9.9997175e-01 2.8235663e-05 4.8027721e-15]
 [4.4554780e-04 9.9218678e-01 7.3676966e-03]
 [1.0000000e+00 2.5645418e-08 5.2755524e-20]]
